### JSONEncoder Class

#### dump
In the previous videos we look at the dump and dumps functions

Beyond the default argument, dump has many other arguments that allow us to control serialization
- **skipkeys**, *bool*, default is False -> if dictionary keys are not basic types (string, int, etc) and skipkeys is set to False we will get a TypeError, otherwise it just skips the key
- **indent**, *int*, default is None -> useful for human readability
- **separators**, *tuple*, defaults to (', ', ': ') -> customizes how the JSON is rendered
- **sort_keys**, *bool*, default is False -> if True, dictionary keys will be sorted
- and many more....( cls for example)

#### The JSONEncoder Class
Python uses an instance of the JSONEncoder class in the json module to serialize data

The JSONEncoder class shares many arguments with the dump / dumps functions 
- default, skipkeys, sort_keys, indent, separators, etc...

The dump and dumps functions hace a cls argument
- allows us to specify our own version of JSONEncoder

#### Why use JSONENcoder at all?
If dump has all the same arguments as JSONEncoder, why use it at all?

To remain consistent in our app, ever time we call dump we need to use the same argument values.

It is easy to make a mistake or forget to specify an argument

Instead, use a custom JSONEncoder  
and just remeber to specify it via the cls argument of dump / dumps

#### How to create a custom JSONEncoder

- subclass JSONEncoder
- custom initialize parent class if we want to
- override the default method 
 - we can handle what we want to ourselves
 - otherwise delegate back to the parent class

Example can be found in code!

#### Code Examples

In [1]:
import json

In [2]:
default_encoder = json.JSONEncoder()

In [4]:
default_encoder.encode([1, 2, 3])

'[1, 2, 3]'

In [5]:
default_encoder.encode((1, 2, 3))

'[1, 2, 3]'

In [6]:
default_encoder.encode(1+1j)

TypeError: Object of type complex is not JSON serializable

In [7]:
from datetime import datetime

In [13]:
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            super().default(arg)

In [14]:
custom_encoder = CustomJSONEncoder()

In [15]:
custom_encoder.encode(True)

'true'

In [16]:
custom_encoder.encode([1, 2, 3])

'[1, 2, 3]'

In [17]:
custom_encoder.encode(datetime.utcnow())

'"2020-11-16T20:06:40.659618"'

In [18]:
custom_encoder.encode({1, 2, 3})

TypeError: Object of type set is not JSON serializable

In [19]:
json.dumps(dict(name='test', time=datetime.utcnow()),
           cls=CustomJSONEncoder
          )

'{"name": "test", "time": "2020-11-16T20:08:25.725037"}'

In [20]:
float('nan')

nan

In [21]:
a = float('nan')

In [22]:
type(a)

float

In [23]:
a

nan

In [24]:
a = float('infinity')

In [25]:
type(a)

float

In [26]:
a

inf

In [27]:
d = {
    'a': float('inf'),
    'b': float('nan')
}

In [30]:
ser = json.dumps(d)

In [33]:
print(ser)

{"a": Infinity, "b": NaN}


In [31]:
deser = json.loads(ser)

In [32]:
deser

{'a': inf, 'b': nan}

In [34]:
json.dumps(d, allow_nan=False)

ValueError: Out of range float values are not JSON compliant

In [39]:
d = {10: "int", 10.5: "float", 1+1j: "complex"}

In [40]:
json.dumps(d)

TypeError: keys must be str, int, float, bool or None, not complex

In [42]:
json.dumps(d, skipkeys=True)

'{"10": "int", "10.5": "float"}'

In [43]:
d = {
    'name': 'Python',
    'age': 27,
    'created_by': 'Guido van Rossum',
    'list': [1, 2, 3]
}

In [44]:
print(json.dumps(d, indent=2))

{
  "name": "Python",
  "age": 27,
  "created_by": "Guido van Rossum",
  "list": [
    1,
    2,
    3
  ]
}


In [45]:
print(json.dumps(d, indent='---'))

{
---"name": "Python",
---"age": 27,
---"created_by": "Guido van Rossum",
---"list": [
------1,
------2,
------3
---]
}


In [47]:
print(json.dumps(d, indent=2, separators=(' - ', ' = ')))

{
  "name" = "Python" - 
  "age" = 27 - 
  "created_by" = "Guido van Rossum" - 
  "list" = [
    1 - 
    2 - 
    3
  ]
}


In [48]:
print(json.dumps(d, indent=2, separators=(', ', ': ')))

{
  "name": "Python", 
  "age": 27, 
  "created_by": "Guido van Rossum", 
  "list": [
    1, 
    2, 
    3
  ]
}


In [49]:
print(json.dumps(d, separators=(', ', ': ')))

{"name": "Python", "age": 27, "created_by": "Guido van Rossum", "list": [1, 2, 3]}


In [50]:
print(json.dumps(d, separators=(',', ':')))

{"name":"Python","age":27,"created_by":"Guido van Rossum","list":[1,2,3]}


In [51]:
print(json.dumps(d, separators=(',', ':'), allow_nan=False, skipkeys=True))

{"name":"Python","age":27,"created_by":"Guido van Rossum","list":[1,2,3]}


In [56]:
class CustomEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        print(kwargs)
        super().__init__(skipkeys=True,
                         allow_nan=False,
                         indent='---',
                         separators=('',' = ')
                        )
    
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            return super().default(arg)

In [57]:
d = {
    'time': datetime.utcnow(),
    1+1j: "complex",
    "name": "Python"
    
}

In [58]:
print(json.dumps(d, cls=CustomEncoder))

{'skipkeys': False, 'ensure_ascii': True, 'check_circular': True, 'allow_nan': True, 'indent': None, 'separators': None, 'default': None, 'sort_keys': False}
{
---"time" = "2020-11-16T20:27:40.342459"
---"name" = "Python"
}


In [59]:
class CustomEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        print(kwargs)
        super().__init__(skipkeys=kwargs['skipkeys'],
                         allow_nan=False,
                         indent='---',
                         separators=('',' = ')
                        )
    
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            return super().default(arg)

In [60]:
d = {
    'time': datetime.utcnow(),
    1+1j: "complex",
    "name": "Python"
    
}

In [61]:
print(json.dumps(d, cls=CustomEncoder))

{'skipkeys': False, 'ensure_ascii': True, 'check_circular': True, 'allow_nan': True, 'indent': None, 'separators': None, 'default': None, 'sort_keys': False}


TypeError: keys must be str, int, float, bool or None, not complex

In [62]:
print(json.dumps(d, cls=CustomEncoder, skipkeys=True))

{'skipkeys': True, 'ensure_ascii': True, 'check_circular': True, 'allow_nan': True, 'indent': None, 'separators': None, 'default': None, 'sort_keys': False}
{
---"time" = "2020-11-16T20:28:45.597708"
---"name" = "Python"
}


In [63]:
class CustomEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        super().__init__(skipkeys=True,
                         allow_nan=False,
                         indent='---',
                         separators=('',' = ')
                        )
    
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            return super().default(arg)

In [64]:
d = {
    'time': datetime.utcnow(),
    1+1j: "complex",
    "name": "Python"
    
}

In [66]:
print(json.dumps(d, cls=CustomEncoder, skipkeys=False))

{
---"time" = "2020-11-16T20:30:20.307214"
---"name" = "Python"
}


In [67]:
print(json.dumps(d, cls=CustomEncoder, skipkeys=True))

{
---"time" = "2020-11-16T20:30:20.307214"
---"name" = "Python"
}


In [68]:
print(json.dumps(d, cls=CustomEncoder))

{
---"time" = "2020-11-16T20:30:20.307214"
---"name" = "Python"
}


In [70]:
class CustomEncoder(json.JSONEncoder):
    def default(self, arg):
        if isinstance(arg, datetime):
            obj = dict(
                datatype="datetime",
                iso=arg.isoformat(),
                date=arg.date().isoformat(),
                time=arg.time().isoformat(),
                year=arg.year,
                month=arg.month,
                day=arg.day,
                hour=arg.hour,
                minutes=arg.minute,
                seconds=arg.second
            )
            return obj
        else:
            return super().default(arg)

In [72]:
d = {'time': datetime.utcnow()}

In [73]:
d

{'time': datetime.datetime(2020, 11, 16, 20, 35, 27, 975069)}

In [77]:
print(json.dumps(d, cls=CustomEncoder, indent=2))

{
  "time": {
    "datatype": "datetime",
    "iso": "2020-11-16T20:35:27.975069",
    "date": "2020-11-16",
    "time": "20:35:27.975069",
    "year": 2020,
    "month": 11,
    "day": 16,
    "hour": 20,
    "minutes": 35,
    "seconds": 27
  }
}
